# Elaborate statistics features for Silvereye

## Dependencies imports


In [ ]:
import xarray as xr
import os
import sys
import pandas as pd
from functools import wraps
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta # $ pip install python-dateutil

In [ ]:
import seaborn as sns  # noqa, pandas aware plotting library

In [ ]:
if ('SP_SRC' in os.environ):
    root_src_dir = os.environ['SP_SRC']
elif sys.platform == 'win32':
    root_src_dir = r'C:\src\csiro\stash\silverpieces'
else:
    root_src_dir = '/home/per202/src/csiro/stash/silverpieces'

pkg_src_dir = root_src_dir
sys.path.append(pkg_src_dir)

In [ ]:
from silverpieces import *

In [ ]:
from silverpieces.functions import *

In [ ]:
dir()

In [ ]:
if ('SP_DATA' in os.environ):
    root_data_dir = os.environ['SP_DATA']
elif sys.platform == 'win32':
    root_data_dir = r'C:\data\silverpieces'
else:
    root_data_dir = '/home/per202/data/silverpieces'


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# the default cmap_sequential for xarray is viridis. 'RdBu' is divergent, but works better for wetness concepts
xr.set_options(cmap_sequential='RdBu')

In [ ]:
# Can get tassie_silo_rain.nc data from https://cloudstor.aarnet.edu.au/plus/s/nj2RevvD1EUD77n
fn = os.path.join(root_data_dir, 'tassie_silo_rain.nc')

In [ ]:
tassie = xr.open_dataset(os.path.join(root_data_dir, 'tassie_silo_rain.nc'))

In [ ]:
tassie

In [ ]:
dr = tassie.daily_rain

In [ ]:
dr.isel(time=4300).plot()

## Use cases

### 3 year period statistic compared to all 3 years periods in the historical record

We want to be able to compare a grid of statistics for a period compared to all periods of similar lengths.
The start and end of the period should be as arbitrary as possible. The sliding window could however be limited or fixed to a year: it is probably moot to compare windows with shifted seasonality. 

#### How does the cumulated rainfall 2016-2018 over TAS compare with all 3 year periods over the record?


In [ ]:
start_time = pd.to_datetime('2016-01-01')
end_time = pd.to_datetime('2018-12-31')

In [ ]:
tmp = dr[:10,:,:]

In [ ]:
tmp

In [ ]:
tmp.rolling(time=3, center=True)

In [ ]:
tmp2 = tmp.rolling(time=3, center=True).construct('tmp')

In [ ]:
tmp2.shape

In [ ]:
(tmp2[:,:,:,0]).plot()

In [ ]:
(tmp2[:,:,:,0] - tmp2[:,:,:,1])[0,:,:].plot()

In [ ]:
tmp.rolling(time=3, center=True).construct('tmp').quantile(.25, dim='tmp')

In [ ]:
dr.isel(time = 1)

In [ ]:
cum_rain = dr.loc[dict(time=slice(start_time, end_time))].sum(dim='time',skipna=False)

In [ ]:
plt.axis('scaled')
cum_rain.plot()

In [ ]:
TIME_DIMNAME = 'time'

In [ ]:
start_time = pd.to_datetime('2007-01-01')
end_time = pd.to_datetime('2009-12-31')

In [ ]:
print(start_time + relativedelta(years=+1))

In [ ]:
cumulated = [dr.loc[dict(time=slice(start_time + relativedelta(years=year), end_time+ relativedelta(years=year)))].sum(dim='time',skipna=False) for year in range(10)]

In [ ]:
cumulated[0]

In [ ]:
three_years_rains = xr.concat(cumulated, dim=TIME_DIMNAME)
three_years_rains.name = 'cum_rain_3yr'

In [ ]:
three_years_rains['time']= [end_time + relativedelta(years=year) for year in range(len(cumulated))]
three_years_rains['time']

In [ ]:
# https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
g_simple = three_years_rains.plot(x='lon', y='lat', col='time', col_wrap=3)

In [ ]:
q = np.array([.1, .5, .9])

In [ ]:
x = np.quantile(three_years_rains, q=q, axis=0, out=None, overwrite_input=False, interpolation='linear', keepdims=False)
x.shape

In [ ]:
three_years_rains.coords, three_years_rains.dims

In [ ]:
y = xr.DataArray(x, 
                coords=[
                    q, 
                    three_years_rains.coords['lat'],
                    three_years_rains.coords['lon']
                ],
                dims=['quantile', 'lat', 'lon'],
                name='quantiles (mm)')

In [ ]:
y

In [ ]:
y.plot(x='lon', y='lat', col='quantile', col_wrap=1, cmap='gist_ncar')


So now we want a map that tells us where the last three years, for every grid cell, sits (which side of every quantile)

In [ ]:
y

In [ ]:
def v_searchsorted(y, z):
    nlat = len(y.lat)
    nlon = len(y.lon)
    xresult = z.copy()
    result = np.empty_like(z, dtype=np.float32)
    for lat in np.arange(nlat):
        for lon in np.arange(nlon):
            zv = z[lat, lon]
            if np.isnan(zv): # just in case - try without?
                result[lat,lon] = np.nan
            else:
                result[lat,lon] = np.searchsorted(y[:,lat, lon].values, zv)
    xresult.values = result
    return xresult

In [ ]:
v_searchsorted(y[:,50:60,50:60], z[50:60,50:60])

In [ ]:
quantile_cat = v_searchsorted(y, z)

In [ ]:
quantile_cat.plot(cmap='viridis_r')

## Appendix

Below are attempts to use Numpy-ic ways to calculate classes. No joy.

In [ ]:
convolve = np.vectorize(np.convolve, signature='(n),(m)->(k)')
convolve(np.eye(2), [1, 2])

In [ ]:
convolve = np.vectorize(np.convolve)
convolve(np.eye(2), [1, 2])

In [ ]:
convolve(np.eye(2), [1, 2])

In [ ]:
z = three_years_rains[-1]

In [ ]:
z.searchsorted(y)

In [ ]:
def searchsorted2d(a,b):
    m,n = a.shape
    max_num = np.maximum(a.max() - a.min(), b.max() - b.min()) + 1
    r = max_num*np.arange(a.shape[0])[:,None]
    p = np.searchsorted( (a+r).ravel(), (b+r).ravel() ).reshape(m,-1)
    return p - n*(np.arange(m)[:,None])

def searchsorted2d(mat,ensembles):
    m,n = mat.shape
    max_num = np.maximum(a.max() - a.min(), b.max() - b.min()) + 1
    r = max_num*np.arange(a.shape[0])[:,None]
    p = np.searchsorted( (a+r).ravel(), (b+r).ravel() ).reshape(m,-1)
    return p - n*(np.arange(m)[:,None])

In [ ]:
np.random.seed(123)
mat = np.sort(np.arange(12)).reshape(3, 4)

In [ ]:
mat = mat.reshape(3, 4)
mat

In [ ]:
mat[0]

In [ ]:
v_searchsorted = np.vectorize(np.searchsorted, signature='(n),(m)->(k)')

In [ ]:
v_searchsorted(mat, np.array([0.5, 5.5, 10.1]))

In [ ]:
v_searchsorted = np.vectorize(np.searchsorted, signature='(n,m),(n,m)->(n)')

In [ ]:
v_searchsorted(mat, np.array([0.5, 5.5]))

In [ ]:
np.searchsorted(np.array([-1, 0, 1, 2]), np.array([-1, 0, 1.1, 2]))

In [ ]:
y.groupby_bins(group, bins, right: bool = True, labels=None, precision: int = 3, include_lowest: bool = False, squeeze: bool = True, restore_coord_dims: Optional[bool] = None)

In [ ]:
z.shape, y.shape

In [ ]:
very_drier = z <= y[0,:,:]

In [ ]:
very_drier.plot()

In [ ]:
z[50,50], y[:,50, 50]

In [ ]:
np.argmin( (200 > y[:,50, 50].values) )

In [ ]:
3000 > y[:,50, 50].values

In [ ]:
np.argmin(z[50,50].values > y[:,50, 50].values)

In [ ]:
np.argmin(z[50,50].values < y[:,50, 50].values)

In [ ]:
(z[50,50].values > y[:,50, 50].values) , (z[50,50].values < y[:,50, 50].values)

In [ ]:
np.eye(4)

In [ ]:
np.argmax(z[50,50].values > y[:,50, 50].values)

In [ ]:
np.argmax(z[50,50].values < y[:,50, 50].values)

In [ ]:
np.searchsorted(y[:,50, 50].values, z[50,50].values)

In [ ]:
np.searchsorted(np.arange(10), 23)

In [ ]:
def searchsorted(a, b):
    func = lambda x, y: np.searchsorted(x[:,,].values, y[].values)
    return xr.apply_ufunc(func, a, b)

In [ ]:
xr.apply_ufunc

In [ ]:
dim = 'time'
xr.apply_ufunc(np.searchsorted,
                   y, z,
                   input_core_dims=[[dim], []])

In [ ]:
import scipy.stats

def earth_mover_distance(first_samples,
                         second_samples,
                         dim='ensemble'):
    return apply_ufunc(scipy.stats.wasserstein_distance,
                       first_samples, second_samples,
                       input_core_dims=[[dim], [dim]],
                       vectorize=True)